5. Usando as bibliotecas Pandas e MatPlotLib e um dataset (podes selecionar das aulas, fazer download na plataforma kaggle ou escolher um dataset pessoal), elabora um notebook jupyter no qual efetues: 
Limpeza e tratamento de dados; 
Processamento de dados: groupby, filter, criação de novas colunas,…; 
Visualização de dados;

Considerações:
Primeiro fazer a importação das bibliotecas e carregar o dataset e exibição das primeiras e ultimas linhas do ficheiro
Seguidamente Limpeza e tratamento dos dados
Posteriormente Processar os dados
Por último Apresentação dos dados


In [ ]:
#importar bibliotecas
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


#Carregamento do dataset e visualização das primeiras e últimas linhas

df = pd.read_csv('titanic.csv')

print(df.head())
print(df.tail())


Verificar o tipo de dados

In [ ]:
print(df.info())

Limpeza e tratamento dos dados
Verificar valores nulos, embora no comando anterior já se via que não existem
Verificar se existem outliers

In [ ]:
print(df.isnull().any())


Criar um novo dataframe (df2) sem a coluna cabin que contém valor nulos

In [ ]:
df2 = df.drop('Cabin',axis=1)
df2.info()

Como  existem valores nulos na coluna 'Age'
Substituir valores nulos na idade, pela média de idades, com este comando 
ver média e mediana da idade

In [ ]:

df2['Age'].fillna(df2['Age'].mean(), inplace=True)
media_idade = df2['Age'].mean()
mediana_idade = df2['Age'].median()

print(media_idade)
print(mediana_idade)


Análise do tipo de embarque com contagem por Id de passageiro

In [ ]:
print(df2.groupby('Embarked')["PassengerId"].count())

Preencher as duas linhas na coluna  'Embarked' com valores nunlos com  o comando:

In [ ]:
df2['Embarked'].fillna('S', inplace=True)
print(df2.info())


Verificar se ainda existem  valores nulos, com o comando seguinte:

In [ ]:

print(df2.isnull().sum())

Criar um ficheiro sem dados nulos

In [68]:
df2.to_csv('titanic_no_nulls.csv', index=False)

Processamento dos dados:
Criar uma nova coluna 'FamilySize' (SibSp + Parch + 1)
Filtrar passageiros com mais de 3 familiares
Agrupar por 'Pclass' e calcular a média de sobrevivência
Exibir a média de sobrevivência por classe


In [ ]:
df2['FamilySize'] = df2['SibSp'] + df2['Parch'] + 1
df_familia_grande = df2[df2['FamilySize'] > 3]
sobrevivencia_por_classe = df2.groupby('Pclass')['Survived'].mean()
print(sobrevivencia_por_classe)

Verificação e eliminação de outliers, se existirem
ler o ficheiro sem valores nulos

In [73]:
df3 = pd.read_csv('titanic_no_nulls.csv')

primeiro passo para identificar outliers é ver uma breve descrição dos dados numéricos

In [ ]:
print(df3.describe())

As colunas a analisar são: "Age", 'SibSp', 'Parch' e 'Fare'
a menor tarifa é negativa(provavelmente um erro)
a maior tarifa é um número 10x maior que o terceiro quartil (75%)
há algém com a idade de 133 anos, hoje pouco provável. deve tratar-se de um outlier ou erro

vamos tratar aqui as anomalias como erros nos dados e vamos remove-los trocando pelo valor médio

In [ ]:
#Análise da idade, primeiros e ultimos 5 valores
print(df3.sort_values('Age', ascending=False).head(5)['Age'])
print(df3.sort_values('Age', ascending=True).head(5)['Age'])

In [76]:
#como a idade de 133 parece um outlier, troca-se pela média de idades
media_idade = df3['Age'].mean()
df3.loc[df3['Age']==133, 'Age'] = media_idade

In [ ]:
#Análise da tarifa, primeiros e últimos 5 valores
print(df3.sort_values('Fare', ascending=False).head(5)['Fare'])
print(df3.sort_values('Fare', ascending=True).head(5)['Fare'])

Constata-se um valor negativo e um valor 10x superior ao segundo mais caro
vamos tratá-los como erros e atualizá-los com o valor da mediana da tarifa
usando a mediana em vez da média, que é menos sensível aos outlires

In [ ]:
mediana_tarifa = df3['Fare'].median()
df3.loc[df3['Fare']>5000, 'Fare']=mediana_tarifa
df3.loc[df3['Fare']<0, 'Fare']=mediana_tarifa

print(df3.sort_values('Fare',ascending=False).head(5)['Fare'])
print(df3.sort_values('Fare',ascending=True).head(5)['Fare'])

Dados estão tratados, guarda-se um novo ficheiro 

In [80]:
df3.to_csv('titanic_no_nulls_no_outliers.csv', index=False)

Apresentação gráfica dos dados sobre:
Sobrevivencia por classe - Gráfico de barras
Idade vs Tarifa vs sobrevivência - Gráfico de dispersão 
Distribuição das idades - Historigrama

In [ ]:
# Gráfico de barras: Sobrevivência por classe
sobrevivencia_por_classe.plot(kind='bar', color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Taxa de Sobrevivência por Classe')
plt.xlabel('Classe')
plt.ylabel('Taxa de Sobrevivência')
plt.xticks(rotation=0)
plt.show()

# Gráfico de dispersão: Idade vs Tarifa, colorido por sobrevivência
plt.scatter(df3['Age'], df3['Fare'], c=df3['Survived'], cmap='coolwarm', alpha=0.6)
plt.title('Idade vs. Tarifa (Colorido por Sobrevivência)')
plt.xlabel('Idade')
plt.ylabel('Tarifa')
plt.colorbar(label='Sobreviveu (1 = Sim, 0 = Não)')
plt.show()

# Histograma: Distribuição de idades
plt.hist(df3['Age'], bins=20, color='orange', edgecolor='black')
plt.title('Distribuição de Idades')
plt.xlabel('Idade')
plt.ylabel('Frequência')
plt.show()